In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
import seaborn as sns
import os
import subprocess


df_bd_preprocesada = pd.read_csv('BD/OnlineRetail_Preprocesada2.csv', sep=',', encoding = 'unicode_escape')
df_pivot = df_bd_preprocesada['InvoiceDate'].str.split(expand = True)
df_pivot.rename(columns = {0: 'Fecha', 1: 'Hora'}, inplace=True)
df_bd_preprocesada = df_bd_preprocesada.drop(columns =['Unnamed: 0','InvoiceDate'])
df_bd_preprocesada = pd.concat([df_bd_preprocesada,df_pivot], axis = 1)
df_bd_preprocesada['Monto'] = df_bd_preprocesada['UnitPrice'] * df_bd_preprocesada['Quantity']
df_bd_ordenadas = df_bd_preprocesada.sort_values('InvoiceNo')

In [2]:
#-------------------------cuantos articulos en promedio se compra en cada transacción---------------------
df_pivot = df_bd_preprocesada[['InvoiceNo','Quantity', 'Monto']].groupby(['InvoiceNo']).sum()
df_pivot=df_pivot.reset_index()
print('El promedio de articulos por compra es  {} ' .format(df_pivot['Quantity'].mean()))
print('la desviación estandar en cantidad de articulos por compra es  {} ' .format(df_pivot['Quantity'].std()))
#-----El resultado es muy alto y poco real, eliminar valores demasiado altos para  mejorar la media-------

El promedio de articulos por compra es  278.6947561501942 
la desviación estandar en cantidad de articulos por compra es  976.4694842812111 


In [3]:
#---------------------Promedio de libras gastadas----------------------------
print('El promedio de libras gastadas por compra es  {} ' .format(df_pivot['Monto'].mean()))
print('la desviación estandar de libras por compra es  {} ' .format(df_pivot['Monto'].std()))
#--------------- Mismo que lo anterior----------------


El promedio de libras gastadas por compra es  479.4566731765214 
la desviación estandar de libras por compra es  1677.916284718865 


In [4]:

#--------------------------Dividir grupo de clientes segun dinero gastado (premium/normal)-------
df_pivot = df_bd_preprocesada[['CustomerID', 'Monto']].groupby(['CustomerID']).sum()
df_pivot=df_pivot.reset_index()
MontoMedio= df_pivot[['Monto']].mean().item()
#----------------Se define monto medio como criterio para separar clientes ----------------------
premium= []
normal = []
for i in range(0, len(df_pivot)):
    if df_pivot.iloc[i]['Monto']<= MontoMedio:
        normal.append(df_pivot.iloc[i]['CustomerID'])
    else:
        premium.append(df_pivot.iloc[i]['CustomerID'])

print("Cantidad total de clientes {}" .format(len(df_pivot)))
print("Cantidad de clientes premium {}" .format(len(premium)))

Cantidad total de clientes 4339
Cantidad de clientes premium 872


In [5]:

#------------------------------Intervalo de menor y mayor numero de transacciones-------------------
df_pivot = df_bd_preprocesada['Hora'].str.split(expand = True, pat =":")
print("la hora donde más se compra es {} horas" .format(df_pivot[0].mode().item()))
df_pivot = df_pivot.groupby([0]).count()
df_pivot=df_pivot.reset_index()
valorMin = df_pivot[1].min().item() #-- Se define valor minimo de transacciones
min = 0
for i in range(0, len(df_pivot)): #--- Se realiza comparación para saber hora de minimás transacciones
    if valorMin == df_pivot.iloc[i][1]:
        min = df_pivot.iloc[i][0]
print("la hora donde menos se compra es {} horas" .format(min))

la hora donde más se compra es 12 horas
la hora donde menos se compra es 6 horas


In [6]:
df_bd_ordenadas.head(392734)

,InvoiceNo,Description,Quantity,UnitPrice,CustomerID,Country,Fecha,Hora,Monto
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,12/1/2010,8:26,15.30
1,536365,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,12/1/2010,8:26,20.34
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,12/1/2010,8:26,22.00
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,12/1/2010,8:26,20.34
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,12/1/2010,8:26,20.34
...,...,...,...,...,...,...,...,...,...
392719,581587,PLASTERS IN TIN STRONGMAN,12,1.65,12680.0,France,12/9/2011,12:50,19.80
392718,581587,PLASTERS IN TIN CIRCUS PARADE,12,1.65,12680.0,France,12/9/2011,12:50,19.80
392730,581587,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,12680.0,France,12/9/2011,12:50,16.60
392723,581587,ALARM CLOCK BAKELIKE IVORY,4,3.75,12680.0,France,12/9/2011,12:50,15.00


In [7]:
len_records = len(df_bd_ordenadas)
print(len_records)

392732


In [37]:
matriz = []
# antes era 22-532-64
maxF = 10
maxL = 532
maxC  = 20
for i in range(maxF):
    a = ["nan"]*maxC
    matriz.append(a)
invoice = df_bd_ordenadas.values[0,0]
ii = 0
jj = 0
for i in range (0,maxL):
    if invoice == df_bd_ordenadas.values[i,0] and jj < maxC:
        desc = str(df_bd_ordenadas.values[i,1])
        matriz[ii][jj]=desc
        jj += 1
    else:
        ii += 1
        if ii > maxF-1:
            i = maxL + 1
        else:
            jj = 1
            desc = str(df_bd_ordenadas.values[i,1])
            matriz[ii][0]=desc
            invoice = df_bd_ordenadas.values[i,0]
print (matriz)    

[['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'RED WOOLLY HOTTIE WHITE HEART.', 'SET 7 BABUSHKA NESTING BOXES', 'GLASS STAR FROSTED T-LIGHT HOLDER', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['RECIPE BOX WITH METAL HEART', 'LOVE BUILDING BLOCK WORD', 'HOME BUILDING BLOCK WORD', 'BOX OF VINTAGE ALPHABET BLOCKS', 'BOX OF VINTAGE JIGSAW BLOCKS ', "POPPY'S PLAYHOUSE BEDROOM ", 'IVORY KNITTED MUG COSY ', 'FELTCRAFT PRINCESS CHARLOTTE DOLL', "POPPY'S PLAYHOUSE KITCHEN", 'DOORMAT NEW ENGLAND', 'ASSORTED COLOUR BIRD ORNAMENT', 'BOX OF 6 ASSORTED COLOUR TEASPOONS', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['BLUE COAT RACK PARIS FASHION', 'YELLOW COAT RACK PARIS 

In [38]:
records=[]
#for i in range(0, maxF):
#    xx=matriz[i]
#    for j in range (0, maxC):
#        records.append(xx[j])
        
#print (records)

In [ ]:
from apyori import apriori
from datetime import datetime
now=datetime.now()
print (now.time())
association_rules = apriori(matriz,min_support=0.0053,min_confidence=0.20,min_lift=3,min_lenght=5)
print (now.time())
association_results=list(association_rules)
now=datetime.now
print (now.time())

15:34:55.691639
15:34:55.691639


In [ ]:
print(len(association_results))

In [81]:
print(association_results[2])

RelationRecord(items=frozenset({'D', ' '}), support=0.005765765765765766, ordered_statistics=[OrderedStatistic(items_base=frozenset({' '}), items_add=frozenset({'D'}), confidence=0.64, lift=111.0), OrderedStatistic(items_base=frozenset({'D'}), items_add=frozenset({' '}), confidence=1.0, lift=111.0)])


In [ ]:
40 productos diferentes